In [2]:
### Script for creating the .pdb files for all salts. ###
import salty
from rdkit import Chem
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.rdmolfiles import MolToPDBFile

data = ['cpt', 'density'] #pull raw data from cpt.csv and density.csv
T = [298, 299] #select narrow state variable ranges
P = [101, 102] #we will set MD simulation to 101 kPa and 298 K
devmodel = salty.aggregate_data(data,T=T,P=P)
print(devmodel.Data_summary)
salts = (devmodel.Data["smiles-cation"]+"."+devmodel.Data["smiles-anion"]).unique() #grab the unique salts

# The counter must be changed manually
i=1
cation = Chem.MolFromSmiles(salts[i].split(".")[0])
anion = Chem.MolFromSmiles(salts[i].split(".")[1]) #grab cation/anion
cation = Chem.AddHs(cation) #add Hs
Chem.EmbedMolecule(cation, Chem.ETKDG()) #Geo optimization for coordinates
Chem.UFFOptimizeMolecule(cation) #FF optimization for better coordinates
anion = Chem.AddHs(anion)
Chem.EmbedMolecule(anion, Chem.ETKDG())
Chem.UFFOptimizeMolecule(anion)
    
if i < 10:
    MolToPDBFile(cation,"C0{}.pdb".format(i)) #write pdb files somewhere
    MolToPDBFile(anion,"A0{}.pdb".format(i)) #with 3 character naming scheme
else:
    MolToPDBFile(cation,"C{}.pdb".format(i)) 
    MolToPDBFile(anion,"A{}.pdb".format(i))

                                                                       0
Unique salts                                                          82
Cations                array(['CCCC[n+]1ccc(cc1)C', 'CCCCCCCCn1cc[n+]...
Anions                 array(['[B-](F)(F)(F)F', 'F[P-](F)(F)(F)(F)F',...
Total datapoints                                                    1630
density                                                   885.3 - 1551.8
cpt                                                       207.5 - 1635.0
Temperature range (K)                                  101.325 - 101.325
Pressure range (kPa)                                       298.1 - 298.2


In [9]:
### Script for pulling out density and heat capacity values which fall within the needed range. ###
import pandas as pd
import csv
import salty

data = pd.read_csv('density.csv', sep=',', header=None)
data.columns = ['NaN','P','Density','NaN','T','Name']
df = pd.DataFrame(data,columns=['NaN','P','Density','NaN','T','Name'])
del df['NaN']

df=df[df['T'] <= 299]
df=df[df['T'] >= 298]
df=df[df['P'] <= 102]
df=df[df['P'] >= 101]

df.to_csv('density_edit.csv', sep=',')

data = pd.read_csv('cpt.csv', sep=',', header=None)
data.columns = ['NaN','cpt','NaN','P','T','Name']
df = pd.DataFrame(data,columns=['NaN','cpt','NaN','P','T','Name'])
del df['NaN']

df=df[df['T'] <= 299]
df=df[df['T'] >= 298]
df=df[df['P'] <= 102]
df=df[df['P'] >= 101]

df.to_csv('cpt_edit.csv', sep=',')

In [ ]:
### Script for making comparision file between SMILES and IUPAC names. ###
import pandas as pd
import csv
import salty

data = pd.read_csv('density.csv', sep=',', header=None)
data.columns = ['NaN','P','Density','NaN','T','Name']
df = pd.DataFrame(data,columns=['NaN','P','Density','NaN','T','Name'])
del df['NaN']

names = data.Name.tolist()
ions = names[0].split(" ")

salts=[0]*len(names)
for i in range(0,len(names)):
    ions=names[i].split(" ",1)
    
    if (len(ions)==1):
        salts[i]="SPACING ERROR"
    else:
        c_smiles=salty.check_name("%s" % ions[0]);
        a_smiles=salty.check_name("%s" % ions[1]);
    
        if (c_smiles==0 or a_smiles==0):
            salts[i]="QUERY NOT FOUND"
        else:
            salts[i]=c_smiles+"."+a_smiles
            
comp = pd.DataFrame({'IUPAC': names, 'SMILES': salts})
comp.to_csv('comparison.csv', sep=',')

In [47]:
### Script for caculating average density from MD calculation result. ###
import pandas as pd
import csv

data=pd.read_csv('80.csv',sep=',',header=None)
data.columns=['time','density']
print(data["density"].mean())
    

1217.2006


Note regarding C55_A55
55,979.7,951.82986,,,979.7,979.7,0,0,2.844762682,824,811.743,,,824,824,0,0,1.4875

In [1]:
pwd

'C:\\Users\\bubbl\\OneDrive\\Desktop\\Summer2018\\Wes\\analysis\\densities'